In [1]:
import numpy as np
from typing import List
import torch

from lib.Tensor import Tensor, force_tensor_method
from lib.NN import Dense, ReLU, Module

In [2]:
class MLP(Module):
    def __init__(self, layer_dims: List[int], activation_fn=ReLU) -> None:
        super().__init__()

        for i in range(len(layer_dims) - 1):
            self.add_module(f"dense_{i}", Dense(layer_dims[i], layer_dims[i+1]))
            if i < len(layer_dims) - 2:
                self.add_module(f"activation_{i}", activation_fn())

    @force_tensor_method
    def forward(self, x: Tensor) -> Tensor:
        for name, module in self._modules.items():
            x = module(x)
        return x

In [3]:
layer_dims = [2, 4, 4, 1]
model = MLP(layer_dims)
input = Tensor(np.random.randn(10, 2), requires_grad=True)

output = model(input)
output

Tensor([[-6.27905561e-08]
 [ 1.61869052e-06]
 [ 2.76903153e-07]
 [-1.46876107e-07]
 [-1.87711920e-07]
 [-9.53964173e-08]
 [-7.33877209e-09]
 [ 1.16244502e-06]
 [-5.82201637e-08]
 [ 2.81100226e-07]], requires_grad=True)

In [5]:
model.list_modules()

['dense_0', 'activation_0', 'dense_1', 'activation_1', 'dense_2']

In [8]:
model.get_module("dense_1").parameters()

[Tensor([[-0.01429236  0.01350345 -0.00396048  0.00135202]
  [-0.01179299 -0.00217827 -0.01334576  0.01822204]
  [ 0.00877019  0.010452    0.01088153  0.00806909]
  [ 0.00295033  0.0070769   0.00321758  0.01096843]], requires_grad=False),
 Tensor([0. 0. 0. 0.], requires_grad=False)]

In [22]:
input_dim = 5
hidden_dim = 10
output_dim = 2
batch_size = 32

layer_dims = [input_dim, hidden_dim, output_dim]

# Create your custom MLP and PyTorch MLP
mlp = MLP(layer_dims, activation_fn=ReLU)
pt_mlp = torch.nn.Sequential(
    torch.nn.Linear(input_dim, hidden_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(hidden_dim, output_dim)
)

with torch.no_grad():
    my_dense_0_params = mlp.get_module("dense_0").parameters()
    pt_mlp[0].weight.copy_(torch.tensor(my_dense_0_params[0].data))
    pt_mlp[0].bias.copy_(torch.tensor(my_dense_0_params[1].data))

    my_dense_1_params = mlp.get_module("dense_1").parameters()
    pt_mlp[2].weight.copy_(torch.tensor(my_dense_1_params[0].data))
    pt_mlp[2].bias.copy_(torch.tensor(my_dense_1_params[1].data))

    # Create random input tensor
    pt_input = torch.randn(batch_size, input_dim)
    my_input = Tensor(pt_input.numpy())  # Assuming you have a way to convert numpy arrays to your Tensor class

    # Forward pass
    pt_output = pt_mlp(pt_input)
    my_output = mlp(my_input)

    print(f"My output: {my_output.data[:2]}")
    print(f"PyTorch output: {pt_output[:2]}")

    assert np.allclose(my_output.data, pt_output.detach().numpy(), atol=1e-6)

My output: [[-1.43804759e-04 -7.80065126e-04]
 [-3.90672927e-05 -1.27185959e-03]]
PyTorch output: tensor([[-1.4380e-04, -7.8007e-04],
        [-3.9067e-05, -1.2719e-03]])


t

In [24]:
def test_mlp():
    input_dim = 5
    hidden_dim = 10
    output_dim = 2
    batch_size = 32

    layer_dims = [input_dim, hidden_dim, output_dim]

    # Create your custom MLP and PyTorch MLP
    mlp = MLP(layer_dims, activation_fn=ReLU)
    pt_mlp = torch.nn.Sequential(
        torch.nn.Linear(input_dim, hidden_dim),
        torch.nn.ReLU(),
        torch.nn.Linear(hidden_dim, output_dim)
    )

    with torch.no_grad():
        my_dense_0_params = mlp.get_module("dense_0").parameters()
        pt_mlp[0].weight.copy_(torch.tensor(my_dense_0_params[0].data))
        pt_mlp[0].bias.copy_(torch.tensor(my_dense_0_params[1].data))

        my_dense_1_params = mlp.get_module("dense_1").parameters()
        pt_mlp[2].weight.copy_(torch.tensor(my_dense_1_params[0].data))
        pt_mlp[2].bias.copy_(torch.tensor(my_dense_1_params[1].data))

        # Create random input tensor
        pt_input = torch.randn(batch_size, input_dim)
        my_input = Tensor(pt_input.numpy())  # Assuming you have a way to convert numpy arrays to your Tensor class

        # Forward pass
        pt_output = pt_mlp(pt_input)
        my_output = mlp(my_input)

        # print(f"My output: {my_output.data[:2]}")
        # print(f"PyTorch output: {pt_output[:2]}")

        assert np.allclose(my_output.data, pt_output.detach().numpy(), atol=1e-6)

test_mlp()